In [1]:
from nlb_tools.nwb_interface import NWBDataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import math
def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    """ Returns the angle in radians between vectors 'v1' and 'v2'::

            >>> angle_between((1, 0, 0), (0, 1, 0))
            1.5707963267948966 (90 deg)
            >>> angle_between((1, 0, 0), (1, 0, 0))
            0.0 (0 deg)
            >>> angle_between((1, 0, 0), (-1, 0, 0))
            3.141592653589793 (180 deg)
    """
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

#Import standard packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import io
from scipy import stats
import pickle

# If you would prefer to load the '.h5' example file rather than the '.pickle' example file. You need the deepdish package
# import deepdish as dd 

#Import function to get the covariate matrix that includes spike history from previous bins
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history

#Import metrics
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

#Import decoder functions
from Neural_Decoding.decoders import WienerCascadeDecoder
from Neural_Decoding.decoders import WienerFilterDecoder
from Neural_Decoding.decoders import DenseNNDecoder
from Neural_Decoding.decoders import SimpleRNNDecoder
from Neural_Decoding.decoders import GRUDecoder
from Neural_Decoding.decoders import LSTMDecoder
from Neural_Decoding.decoders import XGBoostDecoder
from Neural_Decoding.decoders import SVRDecoder

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

def get_sses_pred(y_test,y_test_pred):
    sse=np.sum((y_test_pred-y_test)**2,axis=0)
    return sse

def get_sses_mean(y_test):
    y_mean=np.mean(y_test,axis=0)
    sse_mean=np.sum((y_test-y_mean)**2,axis=0)
    return sse_mean

def nans(shape, dtype=float):
    a = np.empty(shape, dtype)
    a.fill(np.nan)
    return a

def vector_reject(u,v):
    #project u on v, subtract u1 from u
    P = np.outer(v,(v.T))/(v@(v.T))
    u_sub = u - P@u
#     another calculation, to double-check
#     v_norm = np.sqrt(sum(v**2))    
#     proj_u_on_v = (np.dot(u, v)/v_norm**2)*v
#     u_sub = u - proj_u_on_v
    return u_sub

def calc_proj_matrix(A):
    return A@np.linalg.inv(A.T@A)@A.T
def calc_proj(b, A):
    P = calc_proj_matrix(A)
    return P@b.T

# Single Lag

In [2]:
foldername = "~/area2_population_analysis/s1-kinematics/actpas_NWB/"
monkey = "Han_20171207"
filename = foldername + monkey + "_COactpas_TD.nwb"

dataset_5ms = NWBDataset(filename, split_heldout=False)

speed = np.sqrt(np.sum(dataset_5ms.data['hand_vel'][:].T**2,axis=0)).to_numpy().reshape((-1,1))
dataset_5ms.add_continuous_data(speed,'speed')

acceleration = np.diff(speed, axis = 0, prepend=[speed[0]])
dataset_5ms.add_continuous_data(acceleration,'acceleration')

xy_vel = dataset_5ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_5ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_5ms.resample(5)
dataset_5ms.smooth_spk(40, name='smth_40')
# dataset_5ms.smooth_spk(20, name='smth_20')
bin_width = dataset_5ms.bin_width
print(bin_width)

5


In [3]:
# dataset_1ms = NWBDataset(filename, split_heldout=False)
# speed = np.sqrt(np.sum(dataset_1ms.data['hand_vel'][:].T**2,axis=0)).to_numpy().reshape((-1,1))
# dataset_1ms.add_continuous_data(speed,'speed')

In [4]:
n_dims = 20 # for PCA

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_5ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_5ms.data.spikes_smth_40)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_5ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_5ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_5ms.trial_info['cond_dir'] == direction) & (dataset_5ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_5ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

245 trials
153 neurons
(558262, 153)
(558262, 153)
(558262, 20)
PCA total var explained: 0.38767002597161215
[6. 4. 5. 4. 0. 2. 0. 5. 4. 1. 6. 5. 6. 3. 7. 5. 2. 0. 0. 5. 0. 6. 4. 2.
 2. 7. 0. 4. 4. 7. 5. 2. 6. 3. 2. 5. 5. 1. 1. 5. 5. 1. 6. 3. 7. 0. 4. 2.
 1. 5. 1. 1. 1. 1. 1. 6. 2. 0. 5. 4. 7. 6. 3. 1. 5. 3. 0. 4. 6. 6. 0. 1.
 3. 0. 6. 3. 1. 4. 3. 4. 5. 2. 2. 3. 3. 3. 4. 4. 2. 1. 2. 2. 4. 6. 1. 5.
 0. 2. 0. 4. 6. 5. 6. 3. 0. 0. 6. 7. 2. 0. 0. 5. 7. 5. 1. 4. 2. 0. 0. 5.
 4. 6. 3. 6. 3. 6. 3. 1. 7. 3. 3. 7. 5. 5. 3. 1. 5. 0. 7. 4. 7. 5. 5. 1.
 6. 3. 3. 2. 1. 7. 5. 5. 0. 1. 6. 2. 7. 1. 0. 0. 3. 4. 1. 2. 0. 3. 0. 2.
 5. 1. 3. 0. 7. 2. 3. 1. 2. 2. 0. 4. 2. 1. 3. 5. 5. 2. 1. 4. 1. 1. 6. 4.
 1. 4. 7. 0. 2. 5. 6. 2. 1. 1. 5. 7. 1. 1. 2. 2. 5. 0. 6. 0. 0. 0. 3. 7.
 7. 0. 7. 0. 3. 0. 7. 7. 5. 3. 0. 7. 3. 0. 6. 4. 3. 0. 0. 4. 3. 3. 6. 1.
 0. 2. 0. 0. 3.]


In [5]:
# # Prepare for PCA plotting

# active_data = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=(-100, 500), ignored_trials=~trial_mask)
# active_trials_pca = nans([n_trials,n_timepoints,n_dims])
# i = 0
# for idx, trial in active_data.groupby('trial_id'):
#     active_trials_pca[i,:,:]=trial.PCA.to_numpy()
#     i+=1
# print(active_trials_pca.shape)

# plot_dir = np.array([0,45,90,135,180,225,270,315]) # limit plot directions to reduce cluttering
# directions = np.array([0,45,90,135,180,225,270,315])
# pred_range = (-100, 500)
# x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)

# # define some useful time points
# move_idx=0
# ret_idx = 200

# import matplotlib as mpl
# cmap = plt.get_cmap('coolwarm',len(plot_dir))
# custom_palette = [mpl.colors.rgb2hex(cmap(i)) for i in range(len(plot_dir))]

# plot_dims = 10

# fig,ax=plt.subplots(plot_dims,1,figsize=(10,20))
# for i in range(plot_dims):
#     for j in range(len(plot_dir)):
#         color = custom_palette[j]
#         dir_idx = np.argwhere(directions == plot_dir[j])[0]
#         cond_mean_proj = np.mean(active_trials_pca[np.argwhere(cond_dict==dir_idx).flatten(),:,:], axis = 0)[:,i] 
#         pca_mean = np.mean(active_data.PCA.to_numpy(),axis = 0)[i]
#         ax[i].plot(x_axis,cond_mean_proj - pca_mean,linewidth=2.25,color = color,label = plot_dir[j])
        
#         ax[i].axvline(move_idx, color='k',linewidth = .5)
#         ax[i].axvline(ret_idx, color='k',linewidth = .5)
# #         ax[i].set_xlim([0,T])
#         ax[i].set_ylim([-6, 6])
#         ax[i].axhline(0,color ='k',ls = '--')
#         if i<plot_dims-1:
#             ax[i].set_xticks([])
#         else:
#             ax[i].set_xlabel('Time (ms)')
            
#         ax[i].set_yticks([])
#         ax[i].set_ylabel('Dim. '+str(i+1))

#     ax[0].set_title('PCA Projections')
    
# plt.legend(bbox_to_anchor = (1, 1), loc = 'upper left')

In [6]:
def process_train_test(X,y,training_set,test_set):
    X_train = X[training_set,:,:]
    X_test = X[test_set,:,:]
    y_train = y[training_set,:,:]
    y_test = y[test_set,:,:]

    #flat by trials
    X_flat_train = X_train.reshape((X_train.shape[0]*X_train.shape[1]),X_train.shape[2])
    X_flat_test = X_test.reshape((X_test.shape[0]*X_test.shape[1]),X_test.shape[2])
    y_train=y_train.reshape((y_train.shape[0]*y_train.shape[1]),y_train.shape[2])
    y_test=y_test.reshape((y_test.shape[0]*y_test.shape[1]),y_test.shape[2])
    
    X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
    X_flat_train_std=np.nanstd(X_flat_train,axis=0)   
    #array with only 0 will have 0 std and cause errors
    X_flat_train_std[X_flat_train_std==0] = 1
    
    X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
    X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
    y_train_mean=np.mean(y_train,axis=0)
    y_train=y_train-y_train_mean
    y_test=y_test-y_train_mean    
    
    return X_flat_train,X_flat_test,y_train,y_test

In [7]:
def fit_and_predict_weighted(dataset, trial_mask, align_field, align_range, lag, x_field, y_field):
    """Extracts spiking and kinematic data from selected trials and fits linear decoder"""
    # Extract rate data from selected trials
    vel_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~trial_mask)
    # Lag alignment for kinematics and extract kinematics data from selected trials
    lag_align_range = (align_range[0] + lag, align_range[1] + lag)
    rates_df = dataset.make_trial_data(align_field=align_field, align_range=lag_align_range, ignored_trials=~trial_mask)
    
    n_trials = rates_df['trial_id'].nunique()
    n_timepoints = int((align_range[1] - align_range[0])/dataset.bin_width)
    n_neurons = rates_df[x_field].shape[1]
    
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)})
    rates_array = rates_df[x_field].to_numpy()
    vel_array = vel_df[y_field].to_numpy()
    
    vel_array_reshaped = vel_array.reshape(n_trials, n_timepoints, 2)
    sw = 1/((np.std(vel_array_reshaped[:,:,0],axis = 0) + np.std(vel_array_reshaped[:,:,1],axis = 0))/2)
    
    lr_all.fit(rates_array, vel_array,sample_weight = np.tile(sw,n_trials))
    pred_vel = lr_all.predict(rates_array)
    vel_df = pd.concat([vel_df, pd.DataFrame(pred_vel, columns=dataset._make_midx('pred_vel', ['x', 'y'], 2))], axis=1)
#     print(lr_all.best_params_['alpha'])
    
    rates_array = rates_array.reshape(n_trials, n_timepoints, n_neurons)
    vel_array = vel_array_reshaped
    
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,2])
    pred_concat = nans([n_trials*n_timepoints,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(rates_array,vel_array,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)}) 
        lr.fit(X_train, y_train,sample_weight = np.tile(sw,training_set.shape[0]))
        y_test_predicted = lr.predict(X_test)
        
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    print('R2:',R2) 
    return R2, lr_all.best_estimator_.coef_, vel_df

In [8]:
def fit_and_predict(dataset, trial_mask, align_field, align_range, lag, x_field, y_field):
    """Extracts spiking and kinematic data from selected trials and fits linear decoder"""
    # Extract rate data from selected trials
    vel_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~trial_mask)
    # Lag alignment for kinematics and extract kinematics data from selected trials
    lag_align_range = (align_range[0] + lag, align_range[1] + lag)
    rates_df = dataset.make_trial_data(align_field=align_field, align_range=lag_align_range, ignored_trials=~trial_mask)
    
    n_trials = rates_df['trial_id'].nunique()
    n_timepoints = int((align_range[1] - align_range[0])/dataset.bin_width)
    n_neurons = rates_df[x_field].shape[1]
    
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)})
    rates_array = rates_df[x_field].to_numpy()
    vel_array = vel_df[y_field].to_numpy()
    lr_all.fit(rates_array, vel_array)
    pred_vel = lr_all.predict(rates_array)
    vel_df = pd.concat([vel_df, pd.DataFrame(pred_vel, columns=dataset._make_midx('pred_vel', ['x', 'y'], 2))], axis=1)
#     print(lr_all.best_params_['alpha'])
    
    rates_array = rates_array.reshape(n_trials, n_timepoints, n_neurons)
    vel_array = vel_array.reshape(n_trials, n_timepoints, 2)
    
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,2])
    pred_concat = nans([n_trials*n_timepoints,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(rates_array,vel_array,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    print('R2:',R2) 
    return R2, lr_all.best_estimator_.coef_, vel_df

In [9]:
def sub_and_predict(dataset, trial_mask, align_field, align_range, lag, x_field, y_field, weights):
    """Extracts spiking and kinematic data from selected trials and fits linear decoder"""
    # Extract rate data from selected trials
    vel_df = dataset.make_trial_data(align_field=align_field, align_range=align_range, ignored_trials=~trial_mask)
    # Lag alignment for kinematics and extract kinematics data from selected trials
    lag_align_range = (align_range[0] + lag, align_range[1] + lag)
    rates_df = dataset.make_trial_data(align_field=align_field, align_range=lag_align_range, ignored_trials=~trial_mask)
    
    n_trials = rates_df['trial_id'].nunique()
    n_timepoints = int((align_range[1] - align_range[0])/dataset.bin_width)
    n_neurons = rates_df[x_field].shape[1]

    rates_array = rates_df[x_field].to_numpy() - calc_proj(rates_df[x_field].to_numpy(),weights.T).T
    vel_array = vel_df[y_field].to_numpy()
    
    lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)})
    lr_all.fit(rates_array, vel_array)
    pred_vel = lr_all.predict(rates_array)
    vel_df = pd.concat([vel_df, pd.DataFrame(pred_vel, columns=dataset._make_midx('pred_vel', ['x', 'y'], 2))], axis=1)
#     print(lr_all.best_params_['alpha'])
    
    rates_array = rates_array.reshape(n_trials, n_timepoints, n_neurons)
    vel_array = vel_array.reshape(n_trials, n_timepoints, 2)
    
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
    true_concat = nans([n_trials*n_timepoints,2])
    pred_concat = nans([n_trials*n_timepoints,2])
    trial_save_idx = 0
    for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
        #split training and testing by trials
        X_train, X_test, y_train, y_test = process_train_test(rates_array,vel_array,training_set,test_set)
        lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 1, 6)}) 
        lr.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        
        n = y_test_predicted.shape[0]
        true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
        pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
        trial_save_idx += n
    
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    R2 =1-np.sum(sses)/np.sum(sses_mean)     
    print('R2:',R2) 
    return R2, lr_all.best_estimator_.coef_, vel_df

### Timing plots

In [10]:
n_neurons = dataset_5ms.data.spikes.shape[1]
print(n_neurons,'neurons')

active_mask = (~dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')
passive_mask = (dataset_5ms.trial_info.ctr_hold_bump) & (dataset_5ms.trial_info.split != 'none')

figDir = "/Users/sherryan/area2_population_analysis/figures/timing/"

153 neurons


In [11]:
plot_range = (-40, 100)
x_axis = np.arange(plot_range[0], plot_range[1], dataset_5ms.bin_width)
passive_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)
active_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)

plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis, trial.force['y'], color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.force['y'], color='red', linewidth=0.5)
plt.xlabel('Time after bump time (ms)')
plt.ylabel('Force to manipulandum (N)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Force aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_force.png', dpi = 'figure')
plt.close()

In [12]:
plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.speed, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Speed aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_speed.png', dpi = 'figure')
plt.close()

In [13]:
plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.acceleration, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.acceleration, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand acceleration (cm/s^2)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Acceleration aligned to move_onset')
plt.tight_layout()
plt.savefig(figDir + monkey + '_acc.png', dpi = 'figure')
plt.close()

In [14]:
active_trials_spikes = []
for _, trial in active_df.groupby('trial_id'):
    active_trials_spikes.append(np.sum(trial.spikes,axis=1))
passive_trials_spikes = []
for _, trial in passive_df.groupby('trial_id'):
    passive_trials_spikes.append(np.sum(trial.spikes,axis=1))
plt.plot(x_axis,np.sum(active_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(active_trials_spikes)/n_neurons,"-o",color = 'k',label = 'Active')
plt.plot(x_axis,np.sum(passive_trials_spikes,axis = 0)/dataset_5ms.bin_width*1000/len(passive_trials_spikes)/n_neurons,"-o",color = 'red',label = 'Passive')

plt.legend()
plt.title('Peristimulus aligned to move_onset')
plt.ylabel('Trial-average, neuron-average, FR (per sec)')
plt.xlabel('Time (ms)')
plt.axvline(0, color = 'k',linestyle = '--')    
plt.tight_layout()
plt.savefig(figDir + monkey + '_psth.png', dpi = 'figure')
plt.close()

In [15]:
plot_range = (-500, 600)
x_axis = np.arange(plot_range[0], plot_range[1], dataset_5ms.bin_width)
active_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~active_mask)
passive_df = dataset_5ms.make_trial_data(align_field='move_onset_time', align_range=plot_range, ignored_trials=~passive_mask)

plt.figure(figsize=(10,6))
for _, trial in active_df.groupby('trial_id'):
    plt.plot(x_axis,trial.speed, color='k', linewidth=0.5)
for _, trial in passive_df.groupby('trial_id'):
    plt.plot(x_axis, trial.speed, color='red', linewidth=0.5)
plt.xlabel('Time after movement onset (ms)')
plt.ylabel('Hand speed (cm/s)')
plt.axvline(0, color = 'k',linestyle = '--')
plt.title('Speed aligned to move_onset')
plt.axvline(120, color = 'k',linestyle = '--')
plt.tight_layout()
plt.savefig(figDir + monkey + '_speed_whole.png', dpi = 'figure')
plt.close()

## with Neurons

In [ ]:
lag_axis = np.arange(-300,300,20)
x_field = 'spikes_smth_40'
y_field ='hand_acc'
trial_mask = active_mask

# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']

figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"

dim = n_neurons

In [ ]:
ranges = [(-100,500),(0,120),(-100,120),(380,500)]
labels = ['_whole_acc_','_early_acc_','_long_acc_','_late_acc_']

for pred_range, label in zip(ranges, labels):
    x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)
    curr_r2_array = nans([len(lag_axis)])
    curr_coef_array = nans([len(lag_axis),2,dim])
    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
        curr_r2_array[i] = r2
        curr_coef_array[i,:,:] = coef

    idx_max = np.argmax(curr_r2_array)
    time_max = lag_axis[idx_max]
    _, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
    plt.close()

    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
    plt.close()

    plt.plot(lag_axis, curr_r2_array)
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.title('R2 score predicting ' + y_field + ' ' + str(pred_range))
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
    plt.close()

    # ang_dist_to_max = nans([len(lag_axis)])
    # for i in range(0, len(curr_coef_array)):
    #     ang_dist_to_max[i] = math.degrees(angle_between(curr_coef_array[i,0,:],curr_coef_array[idx_max,0,:]))
    # plt.scatter(lag_axis, ang_dist_to_max)
    # plt.title('Angular distance to X-vel decoding dim at t_max')
    # plt.xlabel('Time lag (ms)')
    # plt.ylabel('Angular distance (degrees)')
    # plt.show()

    weights = curr_coef_array[idx_max,:,:]
    for iter in range(0,3):  
        #subtract predictions with primary decoding dimensions (at time with max R2)
        sub_coef_array = nans([len(lag_axis),2,dim])
        sub_r2_array = nans([len(lag_axis)])

        for i in range(len(lag_axis)):
            lag = lag_axis[i]
            r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
            sub_r2_array[i] = r2
            sub_coef_array[i,:,:] = coef

        plt.plot(lag_axis,sub_r2_array)
        plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
        idx_max = np.argmax(sub_r2_array)
        time_max = lag_axis[idx_max]
        plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
        plt.legend()
        plt.xlabel('Time lag (ms)')
        plt.ylabel('R2')
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
        plt.close()

        _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
        for trial_dir, color in zip(plot_dir, colors):
            cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
            for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
                plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        plt.xlabel('Time (ms)')
        plt.ylabel(plot_dim + '_' + y_field)
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
        plt.close()
    #     plt.plot(lag_axis,np.subtract(sub_r2_array,curr_r2_array))
    #     plt.axhline(0,color = 'k',linestyle='--')
    #     plt.title('R2 difference after projecting out t_max dim')
    #     plt.xlabel('Time lag (ms)')
    #     plt.ylabel('R2 difference')
    #     plt.show()
    #     curr_r2_array = sub_r2_array

        #stack the decoding dimensions to be projected out
        weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

## with PCA

In [16]:
x_field = 'PCA'
y_field ='hand_acc'
lag_axis = np.arange(-300,300,20)

# Prepare for plotting
plot_dir = [0.0, 90.0, 180.0, 270.0] # limit plot directions to reduce cluttering
plot_dim = 'x' # plot x velocity
colors = ['red', 'blue', 'green', 'orange']

figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"

dim = n_dims

In [17]:
ranges = [(-100,500),(0,120),(-100,120),(380,500)]
labels = ['_whole_acc_','_early_acc_','_long_acc_','_late_acc_']

for pred_range, label in zip(ranges, labels):
    x_axis = np.arange(pred_range[0], pred_range[1], dataset_5ms.bin_width)
    curr_r2_array = nans([len(lag_axis)])
    curr_coef_array = nans([len(lag_axis),2,dim])
    for i in range(len(lag_axis)):
        lag = lag_axis[i]
        r2, coef, _ = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag, x_field, y_field)
        curr_r2_array[i] = r2
        curr_coef_array[i,:,:] = coef

    idx_max = np.argmax(curr_r2_array)
    time_max = lag_axis[idx_max]
    _, _, vel_df = fit_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field)
    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial[y_field][plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + 'true.png', dpi = 'figure')
    plt.close()

    for trial_dir, color in zip(plot_dir, colors):
        cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
        for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
            plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
    plt.xlabel('Time (ms)')
    plt.ylabel(plot_dim + '_' + y_field)
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'_pred.png', dpi = 'figure')
    plt.close()

    plt.plot(lag_axis, curr_r2_array)
    plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
    plt.legend()
    plt.title('R2 score predicting ' + y_field + ' ' + str(pred_range))
    plt.xlabel('Time lag (ms)')
    plt.ylabel('R2')
    plt.tight_layout()
    plt.savefig(figDir + monkey + label + str(0) +'.png', dpi = 'figure')
    plt.close()

    weights = curr_coef_array[idx_max,:,:]
    for iter in range(0,3):  
        #subtract predictions with primary decoding dimensions (at time with max R2)
        sub_coef_array = nans([len(lag_axis),2,dim])
        sub_r2_array = nans([len(lag_axis)])

        for i in range(len(lag_axis)):
            lag = lag_axis[i]
            r2, coef,_ = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, lag,x_field,y_field,weights)
            sub_r2_array[i] = r2
            sub_coef_array[i,:,:] = coef

        plt.plot(lag_axis,sub_r2_array)
        plt.title('R2 score projecting out #'+ str(iter+1) +' t_max dim')
        idx_max = np.argmax(sub_r2_array)
        time_max = lag_axis[idx_max]
        plt.axvline(time_max, color = 'r', label='t_max = ' + str(time_max))
        plt.legend()
        plt.xlabel('Time lag (ms)')
        plt.ylabel('R2')
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'.png', dpi = 'figure')
        plt.close()

        _, _, vel_df = sub_and_predict(dataset_5ms, trial_mask, 'move_onset_time', pred_range, time_max, x_field, y_field,weights)
        for trial_dir, color in zip(plot_dir, colors):
            cond_ids = dataset_5ms.trial_info[dataset_5ms.trial_info.cond_dir == trial_dir].trial_id
            for _, trial in vel_df[np.isin(vel_df.trial_id, cond_ids)].groupby('trial_id'):
                plt.plot(x_axis, trial.pred_vel[plot_dim], color=color, linewidth=0.5)
        plt.xlabel('Time (ms)')
        plt.ylabel(plot_dim + '_' + y_field)
        plt.tight_layout()
        plt.savefig(figDir + monkey + label + str(iter+1) +'_pred.png', dpi = 'figure')
        plt.close()

        #stack the decoding dimensions to be projected out
        weights = np.vstack((weights,sub_coef_array[idx_max,:,:]))

R2: 0.20108665895930988
R2: 0.2090813595297727
R2: 0.21686707029932228
R2: 0.2287397789986162
R2: 0.24907191576203402
R2: 0.2800789359218361
R2: 0.3206308096301407
R2: 0.3666682980085152
R2: 0.4119342932970589
R2: 0.44989643165416193
R2: 0.476223915889864
R2: 0.49018000003924733
R2: 0.49396087916401765
R2: 0.49076502931710064
R2: 0.4843687079323169
R2: 0.4803141628067714
R2: 0.48448488830043557
R2: 0.49767260675426594
R2: 0.5123928757338442
R2: 0.5173884986747995
R2: 0.5055518819844045
R2: 0.4766909403323176
R2: 0.43491843507004424
R2: 0.38572339710133086
R2: 0.3352378821346095
R2: 0.2898146244557366
R2: 0.25427370049030595
R2: 0.23013860114844198
R2: 0.2156763430458335
R2: 0.20719574109500283
R2: 0.5173884986747995
R2: 0.17931330338298923
R2: 0.18357138503869963
R2: 0.18608345762968603
R2: 0.19164784713430183
R2: 0.20570763615780396
R2: 0.23187807479678157
R2: 0.27034031874639475
R2: 0.31799362098774275
R2: 0.3690533580600882
R2: 0.41647136084369263
R2: 0.4538014939341687
R2: 0.476173

R2: -0.02065423633155361
R2: -0.012173683642760391
R2: 0.002084523543136263
R2: 0.025984519032818842
R2: 0.06070056979993543
R2: 0.09912116579335728
R2: 0.1332018477016601
R2: 0.15933754175747583
R2: 0.1769515500162948
R2: 0.189461913626514
R2: 0.20301165219641437
R2: 0.22095801159338835
R2: 0.2416778462678939
R2: 0.259642126108214
R2: 0.2690842435296523
R2: 0.2673540386293056
R2: 0.25358404573216853
R2: 0.22865814707838972
R2: 0.1986796380962247
R2: 0.17376752881459456
R2: 0.16064785811569515
R2: 0.15873871760244307
R2: 0.1641561034763348
R2: 0.17598077184687066
R2: 0.19735830737607984
R2: 0.2690842435296523
R2: 0.5156874482396526
R2: 0.520468058291486
R2: 0.5314703709410966
R2: 0.5458286133711046
R2: 0.5619669553762936
R2: 0.5808235664858098
R2: 0.6031643854171402
R2: 0.6301507343150561
R2: 0.6618588260253477
R2: 0.6943075299077424
R2: 0.720461083209408
R2: 0.7343980178236041
R2: 0.7329246377175495
R2: 0.7153131693959109
R2: 0.6848070643737694
R2: 0.653428635235352
R2: 0.638658608287

# Multi Lags

In [18]:
dataset_50ms = NWBDataset(filename, split_heldout=False)
xy_vel = dataset_50ms.data['hand_vel'].to_numpy()
xy_acc = np.diff(xy_vel, axis = 0, prepend=[xy_vel[0]])
dataset_50ms.add_continuous_data(xy_acc,'hand_acc',chan_names = ['x','y'])

dataset_50ms.resample(50)
print(dataset_50ms.bin_width)

50


In [19]:
n_dims = 20 # for PCA

active_mask = (~dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')
passive_mask = (dataset_50ms.trial_info.ctr_hold_bump) & (dataset_50ms.trial_info.split != 'none')


trial_mask = active_mask
n_trials = dataset_50ms.trial_info.loc[trial_mask].shape[0]
print(n_trials,'trials')
n_neurons = dataset_50ms.data.spikes.shape[1]
print(n_neurons,'neurons')

all_data = np.array(dataset_50ms.data.spikes)
print(all_data.shape)
data_for_pca = all_data[~np.isnan(all_data).any(axis=1)]
print(data_for_pca.shape)

scaler = StandardScaler()
X = scaler.fit_transform(data_for_pca)
pca = PCA(n_components=n_dims)
X = pca.fit(X)

PCA_data = nans([all_data.shape[0],n_dims])
idx = 0
for dp in all_data:
    dp = dp.reshape((1, -1))
    if np.isnan(dp).any():
        dp_pca = nans([1,n_dims])
    else:
        dp_pca = pca.transform(scaler.transform(dp))
    PCA_data[idx,:] = dp_pca
    idx+=1
print(PCA_data.shape)
dataset_50ms.add_continuous_data(PCA_data,'PCA')
print('PCA total var explained:',sum(pca.explained_variance_ratio_))

245 trials
153 neurons
(55827, 153)
(55827, 153)
(55827, 20)
PCA total var explained: 0.29908653398148444


In [20]:
active_data = dataset_50ms.make_trial_data(align_field='move_onset_time', align_range=(-400, 700), ignored_trials=~trial_mask)
for idx, trial in active_data.groupby('trial_id'):
    n_timepoints = trial.shape[0]
    break
print(n_timepoints,'time bins')

active_trials_neuron = nans([n_trials,n_timepoints,n_neurons])
active_trials_vel = nans([n_trials,n_timepoints,2])
active_trials_acc = nans([n_trials,n_timepoints,2])
active_trials_pca = nans([n_trials,n_timepoints,n_dims])
i = 0
for idx, trial in active_data.groupby('trial_id'):
    active_trials_neuron[i,:,:]=trial.spikes.to_numpy()
    active_trials_vel[i,:,:]=trial.hand_vel.to_numpy()
    active_trials_acc[i,:,:]=trial.hand_acc.to_numpy()
    active_trials_pca[i,:,:]=trial.PCA.to_numpy()
    i+=1
print(active_trials_neuron.shape)
print(active_trials_vel.shape)
print(active_trials_acc.shape)
print(active_trials_pca.shape)

#make dictionary for trial condition (reaching directions) for Stratified CV
active_trials_idx = np.array(dataset_50ms.trial_info.loc[trial_mask]['trial_id'])
cond_dir_idx = []
cond_dict = nans([n_trials])
for direction in [0,45,90,135,180,225,270,315]:
    cond_dir_idx.append(np.where((dataset_50ms.trial_info['cond_dir'] == direction) & (dataset_50ms.trial_info['ctr_hold_bump'] == False) & \
           (dataset_50ms.trial_info['split'] != 'none'))[0])
i = 0
for idx in active_trials_idx:
    for cond in range(0,len(cond_dir_idx)):
        if idx in cond_dir_idx[cond]:
            cond_dict[i] = cond
            break
    i+=1
print(cond_dict)

22 time bins
(245, 22, 153)
(245, 22, 2)
(245, 22, 2)
(245, 22, 20)
[6. 4. 5. 4. 0. 2. 0. 5. 4. 1. 6. 5. 6. 3. 7. 5. 2. 0. 0. 5. 0. 6. 4. 2.
 2. 7. 0. 4. 4. 7. 5. 2. 6. 3. 2. 5. 5. 1. 1. 5. 5. 1. 6. 3. 7. 0. 4. 2.
 1. 5. 1. 1. 1. 1. 1. 6. 2. 0. 5. 4. 7. 6. 3. 1. 5. 3. 0. 4. 6. 6. 0. 1.
 3. 0. 6. 3. 1. 4. 3. 4. 5. 2. 2. 3. 3. 3. 4. 4. 2. 1. 2. 2. 4. 6. 1. 5.
 0. 2. 0. 4. 6. 5. 6. 3. 0. 0. 6. 7. 2. 0. 0. 5. 7. 5. 1. 4. 2. 0. 0. 5.
 4. 6. 3. 6. 3. 6. 3. 1. 7. 3. 3. 7. 5. 5. 3. 1. 5. 0. 7. 4. 7. 5. 5. 1.
 6. 3. 3. 2. 1. 7. 5. 5. 0. 1. 6. 2. 7. 1. 0. 0. 3. 4. 1. 2. 0. 3. 0. 2.
 5. 1. 3. 0. 7. 2. 3. 1. 2. 2. 0. 4. 2. 1. 3. 5. 5. 2. 1. 4. 1. 1. 6. 4.
 1. 4. 7. 0. 2. 5. 6. 2. 1. 1. 5. 7. 1. 1. 2. 2. 5. 0. 6. 0. 0. 0. 3. 7.
 7. 0. 7. 0. 3. 0. 7. 7. 5. 3. 0. 7. 3. 0. 6. 4. 3. 0. 0. 4. 3. 3. 6. 1.
 0. 2. 0. 0. 3.]


### with Neurons

In [25]:
data_range = [-400,700]
figDir = "/Users/sherryan/area2_population_analysis/figures/neurons/act/"
active_x = active_trials_neuron
active_y = active_trials_acc
y_type = 'acceleration'

ranges = [(-100,500),(0,120),(-100,120)]
labels = ['whole_acc','early_acc','long_acc']

dim = n_neurons

In [26]:
for pred_range, label in zip(ranges, labels):

    idx1 = int((pred_range[0] - data_range[0])/dataset_50ms.bin_width)
    idx2 = int(n_timepoints - (data_range[1]-pred_range[1])/dataset_50ms.bin_width)
    
    if pred_range == (-100,500):
        t_before_range = range(0,201,50);
        t_after_range = range(0,201,50);        
    else:        
        t_before_range = range(0,301,50);
        t_after_range = range(0,501,50);

    multi_R2s = nans([len(t_before_range),len(t_after_range)])
    multi_coefs = []
    j,k=0,0
    for time_before in t_before_range:
        coef_arr = []
        for time_after in t_after_range:
            print('Predicting with',-time_before, 'to', time_after,'ms neural data')

            bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
            bins_current= 1 #Whether to use concurrent time bin of neural data
            bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

            n_total_bins = bins_before + bins_current + bins_after

            X =  nans([n_trials,idx2-idx1,n_total_bins*dim])
            i = 0
            for trial_data in active_x:
                trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
                trial_hist = trial_hist[idx1:idx2,:,:]
                trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
                X[i,:,:] = trial_hist_flat
                i+=1
            y = active_y[:,idx1:idx2,:]

            lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
            X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
            y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
            lr_all.fit(X_reshaped, y_reshaped)
            print(lr_all.best_params_['alpha'])

            skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
            true_concat = nans([(n_trials*(idx2-idx1)),2])
            pred_concat = nans([(n_trials*(idx2-idx1)),2])
            trial_save_idx = 0
            for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
                #split training and testing by trials
                X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
                lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
                lr.fit(X_train, y_train)
                y_test_predicted = lr.predict(X_test)
                n = y_test_predicted.shape[0]
                true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
                pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
                trial_save_idx += n

            sses =get_sses_pred(true_concat,pred_concat)
            sses_mean=get_sses_mean(true_concat)
            multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
            print('R2:',multi_R2s[j,k])
            coef_arr.append(lr_all.best_estimator_.coef_)
            k += 1
        j += 1
        k = 0
        multi_coefs.append(coef_arr)

    fig, ax = plt.subplots()
    im = ax.imshow(multi_R2s)
    ax.set_xlabel('Length of lagging info')
    ax.set_ylabel('Length of leading info')

    ax.set_xticks(np.arange(len(t_after_range)))
    ax.set_yticks(np.arange(len(t_before_range)))
    ax.set_xticklabels(labels=t_after_range)
    ax.set_yticklabels(labels=t_before_range)

    ax.set_title('R2 predicting ' + str(pred_range) + ' ' + y_type +'\nwith different lagging/leading info')
    fig.tight_layout()

    for i in range(len(t_before_range)):
        for j in range(len(t_after_range)):
            text = ax.text(j, i, str(int(multi_R2s[i, j]*1000)/1000),
                           ha="center", va="center", color="w")

    plt.tight_layout()
    plt.savefig(figDir + monkey + '_multi_' + label + '.png', dpi = 'figure')
    plt.close()
    
    if pred_range == (-100,500):
        coef_X = multi_coefs[-1][-1][0] #which entry's weights to use

        t_label = np.arange(-200,201,50)

        n_weights = len(t_before_range) + len(t_after_range) - 1
        coef_X_reshaped = coef_X.reshape(n_weights,n_neurons)
        angDist_array = nans([n_weights,n_weights])
        for i in range(n_weights):
            for j in range(n_weights):
                angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
        fig, ax = plt.subplots(figsize=(6, 6))
        im = ax.imshow(angDist_array)
        ax.set_xlabel('Bin time (ms)')
        ax.set_ylabel('Bin time (ms)')

        ax.set_xticks(np.arange(len(t_label)))
        ax.set_yticks(np.arange(len(t_label)))
        ax.set_xticklabels(labels=t_label)
        ax.set_yticklabels(labels=t_label)

        ax.set_title("Angle between weight vectors at time points")
        fig.tight_layout()

        for i in range(len(t_label)):
            for j in range(len(t_label)):
                text = ax.text(j, i, str(int(angDist_array[i, j])),
                               ha="center", va="center", color="w")
        plt.tight_layout()
        plt.savefig(figDir + monkey + '_multi_' + label + '_deg.png', dpi = 'figure')
        plt.close()

Predicting with 0 to 0 ms neural data
100.0
R2: 0.5145890864490528
Predicting with 0 to 50 ms neural data
1000.0
R2: 0.6327963509369792
Predicting with 0 to 100 ms neural data
1000.0
R2: 0.67911719800766
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.7020884778770538
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.7055130378453773
Predicting with -50 to 0 ms neural data
100.0
R2: 0.6484463741843767
Predicting with -50 to 50 ms neural data
1000.0
R2: 0.7175345211020209
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.7470871011387126
Predicting with -50 to 150 ms neural data
1000.0
R2: 0.7600613455762663
Predicting with -50 to 200 ms neural data
1000.0
R2: 0.7611620080754082
Predicting with -100 to 0 ms neural data
1000.0
R2: 0.6960876342971795
Predicting with -100 to 50 ms neural data
1000.0
R2: 0.7512553458047537
Predicting with -100 to 100 ms neural data
1000.0
R2: 0.7734026565864731
Predicting with -100 to 150 ms neural data
1000.0
R2: 0.7832676088575995
Predicting 

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2901737600.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2901737600.py:118: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Predicting with 0 to 0 ms neural data
100.0
R2: 0.7213265993242554
Predicting with 0 to 50 ms neural data
100.0
R2: 0.7716100364194546
Predicting with 0 to 100 ms neural data
1000.0
R2: 0.7969129036794693
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.8102856693595266
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.8201484615247602
Predicting with 0 to 250 ms neural data
1000.0
R2: 0.8296951101698345
Predicting with 0 to 300 ms neural data
1000.0
R2: 0.832645379163077
Predicting with 0 to 350 ms neural data
1000.0
R2: 0.8378029387481037
Predicting with 0 to 400 ms neural data
1000.0
R2: 0.8409266219188153
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.8439431485270403
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.8426940784686194
Predicting with -50 to 0 ms neural data
100.0
R2: 0.7683187129818618
Predicting with -50 to 50 ms neural data
100.0
R2: 0.8157935241181382
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.8206683504583127
Predicting with -50 to 15

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2901737600.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Predicting with 0 to 0 ms neural data
100.0
R2: 0.634916809385799
Predicting with 0 to 50 ms neural data
100.0
R2: 0.7385455882836808
Predicting with 0 to 100 ms neural data
1000.0
R2: 0.771011657510292
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.7911035639114187
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.801315822652757
Predicting with 0 to 250 ms neural data
1000.0
R2: 0.8077214906590353
Predicting with 0 to 300 ms neural data
1000.0
R2: 0.8065332137242224
Predicting with 0 to 350 ms neural data
1000.0
R2: 0.8111170076132383
Predicting with 0 to 400 ms neural data
1000.0
R2: 0.8112888926328612
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.8117366473672094
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.8113560759783573
Predicting with -50 to 0 ms neural data
100.0
R2: 0.7242287869004848
Predicting with -50 to 50 ms neural data
1000.0
R2: 0.792674558153223
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.8094892378892304
Predicting with -50 to 150 

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2901737600.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


### with PCA

In [27]:
data_range = [-400,700]
figDir = "/Users/sherryan/area2_population_analysis/figures/PCA/act/"
active_x = active_trials_pca
active_y = active_trials_acc
y_type = 'acceleration'

ranges = [(-100,500),(0,120),(-100,120)]
labels = ['whole_acc','early_acc','long_acc']

dim = n_dims

In [29]:
for pred_range, label in zip(ranges, labels):

    idx1 = int((pred_range[0] - data_range[0])/dataset_50ms.bin_width)
    idx2 = int(n_timepoints - (data_range[1]-pred_range[1])/dataset_50ms.bin_width)

    if pred_range == (-100,500):
        t_before_range = range(0,201,50);
        t_after_range = range(0,201,50);        
    else:        
        t_before_range = range(0,301,50);
        t_after_range = range(0,501,50);

    multi_R2s = nans([len(t_before_range),len(t_after_range)])
    multi_coefs = []
    j,k=0,0
    for time_before in t_before_range:
        coef_arr = []
        for time_after in t_after_range:
            print('Predicting with',-time_before, 'to', time_after,'ms neural data')

            bins_before= int(time_before/dataset_50ms.bin_width) #How many bins of neural data prior to the output are used for decoding
            bins_current= 1 #Whether to use concurrent time bin of neural data
            bins_after= int(time_after/dataset_50ms.bin_width) #How many bins of neural data after the output are used for decoding

            n_total_bins = bins_before + bins_current + bins_after

            X =  nans([n_trials,idx2-idx1,n_total_bins*dim])
            i = 0
            for trial_data in active_x:
                trial_hist=get_spikes_with_history(trial_data,bins_before,bins_after,bins_current)
                trial_hist = trial_hist[idx1:idx2,:,:]
                trial_hist_flat=trial_hist.reshape(trial_hist.shape[0],(trial_hist.shape[1]*trial_hist.shape[2]))
                X[i,:,:] = trial_hist_flat
                i+=1
            y = active_y[:,idx1:idx2,:]

            lr_all = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)})
            X_reshaped = X.reshape((X.shape[0]*X.shape[1]),X.shape[2])
            y_reshaped = y.reshape((y.shape[0]*y.shape[1]),y.shape[2])
            lr_all.fit(X_reshaped, y_reshaped)
            print(lr_all.best_params_['alpha'])

            skf = StratifiedKFold(n_splits=5,shuffle=True,random_state = 42)   
            true_concat = nans([(n_trials*(idx2-idx1)),2])
            pred_concat = nans([(n_trials*(idx2-idx1)),2])
            trial_save_idx = 0
            for training_set, test_set in skf.split(range(0,n_trials),cond_dict):
                #split training and testing by trials
                X_train, X_test, y_train, y_test = process_train_test(X,y,training_set,test_set)
                lr = GridSearchCV(Ridge(), {'alpha': np.logspace(-4, 4, 9)}) 
                lr.fit(X_train, y_train)
                y_test_predicted = lr.predict(X_test)
                n = y_test_predicted.shape[0]
                true_concat[trial_save_idx:trial_save_idx+n,:] = y_test
                pred_concat[trial_save_idx:trial_save_idx+n,:] = y_test_predicted
                trial_save_idx += n

            sses =get_sses_pred(true_concat,pred_concat)
            sses_mean=get_sses_mean(true_concat)
            multi_R2s[j,k] =1-np.sum(sses)/np.sum(sses_mean)     
            print('R2:',multi_R2s[j,k])
            coef_arr.append(lr_all.best_estimator_.coef_)
            k += 1
        j += 1
        k = 0
        multi_coefs.append(coef_arr)

    fig, ax = plt.subplots()
    im = ax.imshow(multi_R2s)
    ax.set_xlabel('Length of lagging info')
    ax.set_ylabel('Length of leading info')

    ax.set_xticks(np.arange(len(t_after_range)))
    ax.set_yticks(np.arange(len(t_before_range)))
    ax.set_xticklabels(labels=t_after_range)
    ax.set_yticklabels(labels=t_before_range)

    ax.set_title('R2 predicting ' + str(pred_range) + ' ' + y_type +'\nwith different lagging/leading info')
    fig.tight_layout()

    for i in range(len(t_before_range)):
        for j in range(len(t_after_range)):
            text = ax.text(j, i, str(int(multi_R2s[i, j]*1000)/1000),
                           ha="center", va="center", color="w")

    plt.tight_layout()
    plt.savefig(figDir + monkey + '_multi_' + label + '.png', dpi = 'figure')
    plt.close()
    
    if pred_range == (-100,500):
        coef_X = multi_coefs[-1][-1][0] #which entry's weights to use

        t_label = np.arange(-200,201,50)

        n_weights = len(t_before_range) + len(t_after_range) - 1
        coef_X_reshaped = coef_X.reshape(n_weights,dim)
        angDist_array = nans([n_weights,n_weights])
        for i in range(n_weights):
            for j in range(n_weights):
                angDist_array[i,j] = math.degrees(angle_between(coef_X_reshaped[i,:],coef_X_reshaped[j,:]))
        fig, ax = plt.subplots(figsize=(6, 6))
        im = ax.imshow(angDist_array)
        ax.set_xlabel('Bin time (ms)')
        ax.set_ylabel('Bin time (ms)')

        ax.set_xticks(np.arange(len(t_label)))
        ax.set_yticks(np.arange(len(t_label)))
        ax.set_xticklabels(labels=t_label)
        ax.set_yticklabels(labels=t_label)

        ax.set_title("Angle between weight vectors at time points")
        fig.tight_layout()

        for i in range(len(t_label)):
            for j in range(len(t_label)):
                text = ax.text(j, i, str(int(angDist_array[i, j])),
                               ha="center", va="center", color="w")
        plt.tight_layout()
        plt.savefig(figDir + monkey + '_multi_' + label + '_deg.png', dpi = 'figure')
        plt.close()

Predicting with 0 to 0 ms neural data
100.0
R2: 0.3958884482843952
Predicting with 0 to 50 ms neural data
100.0
R2: 0.5365462117942983
Predicting with 0 to 100 ms neural data
100.0
R2: 0.5988363256587734
Predicting with 0 to 150 ms neural data
1000.0
R2: 0.6190140261692847
Predicting with 0 to 200 ms neural data
1000.0
R2: 0.6248824331427091
Predicting with -50 to 0 ms neural data
100.0
R2: 0.5274648284881696
Predicting with -50 to 50 ms neural data
100.0
R2: 0.6345473452486661
Predicting with -50 to 100 ms neural data
1000.0
R2: 0.6826223033738286
Predicting with -50 to 150 ms neural data
1000.0
R2: 0.6969004749488928
Predicting with -50 to 200 ms neural data
1000.0
R2: 0.7002040415713553
Predicting with -100 to 0 ms neural data
1000.0
R2: 0.5822558844353678
Predicting with -100 to 50 ms neural data
1000.0
R2: 0.6793911565006001
Predicting with -100 to 100 ms neural data
1000.0
R2: 0.7175055294300794
Predicting with -100 to 150 ms neural data
1000.0
R2: 0.7297660119465703
Predicting w

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2187432194.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2187432194.py:118: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Predicting with 0 to 0 ms neural data
100.0
R2: 0.6668253096118432
Predicting with 0 to 50 ms neural data
100.0
R2: 0.7651836876885119
Predicting with 0 to 100 ms neural data
100.0
R2: 0.8006094581241301
Predicting with 0 to 150 ms neural data
100.0
R2: 0.8199979963601481
Predicting with 0 to 200 ms neural data
100.0
R2: 0.8271620117814726
Predicting with 0 to 250 ms neural data
1000.0
R2: 0.8273195508662501
Predicting with 0 to 300 ms neural data
1000.0
R2: 0.8241979391434583
Predicting with 0 to 350 ms neural data
1000.0
R2: 0.8219335979604836
Predicting with 0 to 400 ms neural data
1000.0
R2: 0.8250704338085685
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.8241634767947358
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.8243250127338069
Predicting with -50 to 0 ms neural data
100.0
R2: 0.7490391785826123
Predicting with -50 to 50 ms neural data
100.0
R2: 0.8218249590095474
Predicting with -50 to 100 ms neural data
100.0
R2: 0.83771130299429
Predicting with -50 to 150 ms 

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2187432194.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()


Predicting with 0 to 0 ms neural data
100.0
R2: 0.5384370177098068
Predicting with 0 to 50 ms neural data
100.0
R2: 0.6978155929156001
Predicting with 0 to 100 ms neural data
100.0
R2: 0.7447969162204449
Predicting with 0 to 150 ms neural data
100.0
R2: 0.7660487542240194
Predicting with 0 to 200 ms neural data
100.0
R2: 0.7743840807322036
Predicting with 0 to 250 ms neural data
1000.0
R2: 0.7796836721055256
Predicting with 0 to 300 ms neural data
1000.0
R2: 0.7792971229182712
Predicting with 0 to 350 ms neural data
1000.0
R2: 0.7779999320172319
Predicting with 0 to 400 ms neural data
1000.0
R2: 0.7787239332572395
Predicting with 0 to 450 ms neural data
1000.0
R2: 0.7757627440307958
Predicting with 0 to 500 ms neural data
1000.0
R2: 0.7735717006819911
Predicting with -50 to 0 ms neural data
100.0
R2: 0.6556795555973476
Predicting with -50 to 50 ms neural data
100.0
R2: 0.76027197682565
Predicting with -50 to 100 ms neural data
100.0
R2: 0.7882696869817984
Predicting with -50 to 150 ms 

/var/folders/c_/3p5f6szx247fkf6ftdgp147w0000gn/T/ipykernel_6520/2187432194.py:86: UserWarning: The figure layout has changed to tight
  plt.tight_layout()
